### 🚀 Paste Your **Ngrok Authtoken** in the Cell Below and Press **Enter**

---

📘 **Step-by-Step Guide:**  
[GitHub Repository – Ngrok Authtoken Creation](https://github.com/aidailyhub74/ngrok-authtoken-creation)

🎥 **Tutorial Videos:**  
1. **Create Ngrok Account:** [Watch on YouTube](https://www.youtube.com/watch?v=9CL1_QnUnvc)  
2. **Use Ngrok Key in Colab (Run Streamlit App):** [Watch on YouTube](https://www.youtube.com/watch?v=3qvuY-EY8to)

In [1]:
import getpass
ngrok_key = getpass.getpass("Enter ngrok key: ")

Enter ngrok key: ··········


In [2]:
!pip install streamlit pyngrok torchxrayvision

In [3]:
!pip install langchain-google-genai==2.1.10 langchain-tavily==0.2.11 langgraph==0.6.7 langchain-core==0.3.75

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: langgraph-checkpoint
    Found existing installation: langgraph-checkpoint 3.0.0
    Uninstalling langgraph-checkpoint-3.0.0:
      Successfully uninstalled langgraph-checkpoint-3.0.0
  Attempting uninstall: langgraph-prebuilt
    Found existing installation: langgraph-prebuilt 1.0.1
    Uninstalling langgraph-prebuilt-1.0.1:
      Successfully uninstalled langgraph-prebuilt-1.0.1
  Attempting uninstall: langchain
    Found existing installation: langchain 1.0.0
    Uninstalling langchain-1.0.0:
      Successfully uninstalled langchain-1.0.0
  Attempting uninstall: langgraph
    Found existing installation: langgraph 1.0.1
    Uninstalling langgraph-1.0.1:
      Successfully uninstalled langgraph-1.0.1
  Attempting uninstall: langc

In [3]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [5]:
!ollama --version

ollama version is 0.12.6


In [4]:
import subprocess

subprocess.Popen(["ollama", "serve"])
import time

time.sleep(3)

In [6]:
!ollama pull amsaravi/medgemma-4b-it:q8

In [6]:
!ollama list

NAME                          ID              SIZE      MODIFIED       
amsaravi/medgemma-4b-it:q8    905dc0be940a    5.0 GB    17 minutes ago    


In [7]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key: ")

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

Tavily API key: ··········
Enter API key for Google Gemini: ··········


In [8]:
%%writefile helper.py
import warnings
warnings.filterwarnings("ignore")
import base64
from typing import Optional, Dict, Tuple, Type, Any
from PIL import Image
from openai import OpenAI
from pydantic import BaseModel, Field
from langchain.tools.base import BaseTool
from langchain.callbacks.manager import CallbackManagerForToolRun, AsyncCallbackManagerForToolRun
from langchain_tavily import TavilySearch
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import InMemorySaver
from typing import Dict, Optional, Tuple, Type
from pydantic import BaseModel, Field

import skimage.io
import torch
import torchvision
import torchxrayvision as xrv

from langchain_core.callbacks import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain_core.tools import BaseTool
from typing import Dict, List, Optional, Tuple, Type, Any
from pathlib import Path
import uuid
import tempfile

import numpy as np
import torch
import torchvision
import torchxrayvision as xrv
import matplotlib.pyplot as plt
import skimage.io
import skimage.measure
import skimage.transform
import traceback

from pydantic import BaseModel, Field
from langchain_core.callbacks import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain_core.tools import BaseTool


class ChestXRaySegmentationInput(BaseModel):
    """Input schema for the Chest X-ray Segmentation Tool."""

    image_path: str = Field(..., description="Path to the chest X-ray image file to be segmented")
    organs: Optional[List[str]] = Field(
        None,
        description="List of organs to segment. If None, all available organs will be segmented. "
        "Available organs: Left/Right Clavicle, Left/Right Scapula, Left/Right Lung, "
        "Left/Right Hilus Pulmonis, Heart, Aorta, Facies Diaphragmatica, "
        "Mediastinum, Weasand, Spine",
    )


class OrganMetrics(BaseModel):
    """Detailed metrics for a segmented organ."""

    # Basic metrics
    area_pixels: int = Field(..., description="Area in pixels")
    area_cm2: float = Field(..., description="Approximate area in cm²")
    centroid: Tuple[float, float] = Field(..., description="(y, x) coordinates of centroid")
    bbox: Tuple[int, int, int, int] = Field(
        ..., description="Bounding box coordinates (min_y, min_x, max_y, max_x)"
    )

    # Size metrics
    width: int = Field(..., description="Width of the organ in pixels")
    height: int = Field(..., description="Height of the organ in pixels")
    aspect_ratio: float = Field(..., description="Height/width ratio")

    # Position metrics
    relative_position: Dict[str, float] = Field(
        ..., description="Position relative to image boundaries (0-1 scale)"
    )

    # Analysis metrics
    mean_intensity: float = Field(..., description="Mean pixel intensity in the organ region")
    std_intensity: float = Field(..., description="Standard deviation of pixel intensity")
    confidence_score: float = Field(..., description="Model confidence score for this organ")


class ChestXRaySegmentationTool(BaseTool):
    """Tool for anatomical segmentation of chest X-ray images.

    This tool performs segmentation of chest X-ray images into multiple
    anatomical structures. It provides both a visualization of the segmented
    regions and quantitative metrics for further analysis.

    Supported anatomical structures include:
    - Left/Right Clavicle (collarbones)
    - Left/Right Scapula (shoulder blades)
    - Left/Right Lung
    - Left/Right Hilus Pulmonis (lung roots)
    - Heart
    - Aorta
    - Facies Diaphragmatica (diaphragm)
    - Mediastinum (central cavity)
    - Weasand (esophagus)
    - Spine

    Note: Segmentation area estimates may be less accurate for non-DICOM images.
    """

    name: str = "chest_xray_segmentation"
    description: str = (
        "Performs anatomical segmentation on chest X-ray images and outputs both "
        "a segmentation visualization and quantitative metrics. "
        "Supported structures: Left/Right Clavicle, Left/Right Scapula, Left/Right Lung, "
        "Left/Right Hilus Pulmonis, Heart, Aorta, Facies Diaphragmatica, Mediastinum, "
        "Weasand (esophagus), and Spine. "
        "Note: Accuracy of area estimation is reduced if the input is not a DICOM image."
    )
    args_schema: Type[BaseModel] = ChestXRaySegmentationInput

    model: Optional[object] = None
    device: Optional[str] = "cuda"
    transform: Optional[object] = None
    pixel_spacing_mm: float = 0.2
    temp_dir: Path = Path("temp")
    organ_map: Optional[Dict[str, int]] = None

    def __init__(self, device: Optional[str] = "cuda", temp_dir: Optional[Path] = Path("temp")):
        """Initialize the segmentation tool with model and temporary directory."""
        super().__init__()
        self.model = xrv.baseline_models.chestx_det.PSPNet()
        self.device = torch.device(device) if device else "cuda"
        self.model = self.model.to(self.device)
        self.model.eval()

        self.transform = torchvision.transforms.Compose(
            [xrv.datasets.XRayCenterCrop(), xrv.datasets.XRayResizer(512)]
        )

        self.temp_dir = temp_dir if isinstance(temp_dir, Path) else Path(temp_dir)
        self.temp_dir.mkdir(exist_ok=True)

        # Map friendly names to model target indices
        self.organ_map = {
            "Left Clavicle": 0,
            "Right Clavicle": 1,
            "Left Scapula": 2,
            "Right Scapula": 3,
            "Left Lung": 4,
            "Right Lung": 5,
            "Left Hilus Pulmonis": 6,
            "Right Hilus Pulmonis": 7,
            "Heart": 8,
            "Aorta": 9,
            "Facies Diaphragmatica": 10,
            "Mediastinum": 11,
            "Weasand": 12,
            "Spine": 13,
        }

    def _align_mask_to_original(
        self, mask: np.ndarray, original_shape: Tuple[int, int]
    ) -> np.ndarray:
        """
        Align a mask from the transformed (cropped/resized) space back to the full original image.
        Assumes that the transform does a center crop to a square of side = min(original height, width)
        and then resizes to (512,512).
        """
        orig_h, orig_w = original_shape
        crop_size = min(orig_h, orig_w)
        crop_top = (orig_h - crop_size) // 2
        crop_left = (orig_w - crop_size) // 2

        # Resize mask (from 512x512) to the cropped region size
        resized_mask = skimage.transform.resize(
            mask, (crop_size, crop_size), order=0, preserve_range=True, anti_aliasing=False
        )
        full_mask = np.zeros(original_shape)
        full_mask[crop_top : crop_top + crop_size, crop_left : crop_left + crop_size] = resized_mask
        return full_mask

    def _compute_organ_metrics(
        self, mask: np.ndarray, original_img: np.ndarray, confidence: float
    ) -> Optional[OrganMetrics]:
        """Compute comprehensive metrics for a single organ mask."""
        # Align mask to the original image coordinates if needed
        if mask.shape != original_img.shape:
            mask = self._align_mask_to_original(mask, original_img.shape)

        props = skimage.measure.regionprops(mask.astype(int))
        if not props:
            return None

        props = props[0]
        area_cm2 = mask.sum() * (self.pixel_spacing_mm / 10) ** 2

        img_height, img_width = mask.shape
        cy, cx = props.centroid
        relative_pos = {
            "top": cy / img_height,
            "left": cx / img_width,
            "center_dist": np.sqrt(((cy / img_height - 0.5) ** 2 + (cx / img_width - 0.5) ** 2)),
        }

        organ_pixels = original_img[mask > 0]
        mean_intensity = organ_pixels.mean() if len(organ_pixels) > 0 else 0
        std_intensity = organ_pixels.std() if len(organ_pixels) > 0 else 0

        return OrganMetrics(
            area_pixels=int(mask.sum()),
            area_cm2=float(area_cm2),
            centroid=(float(cy), float(cx)),
            bbox=tuple(map(int, props.bbox)),
            width=int(props.bbox[3] - props.bbox[1]),
            height=int(props.bbox[2] - props.bbox[0]),
            aspect_ratio=float(
                (props.bbox[2] - props.bbox[0]) / max(1, props.bbox[3] - props.bbox[1])
            ),
            relative_position=relative_pos,
            mean_intensity=float(mean_intensity),
            std_intensity=float(std_intensity),
            confidence_score=float(confidence),
        )

    def _save_visualization(
        self, original_img: np.ndarray, pred_masks: torch.Tensor, organ_indices: List[int]
    ) -> str:
        """Save visualization of original image with segmentation masks overlaid."""
        plt.figure(figsize=(10, 10))
        plt.imshow(
            original_img, cmap="gray", extent=[0, original_img.shape[1], original_img.shape[0], 0]
        )

        # Generate color palette for organs
        colors = plt.cm.rainbow(np.linspace(0, 1, len(organ_indices)))

        # Process and overlay each organ mask
        for idx, (organ_idx, color) in enumerate(zip(organ_indices, colors)):
            mask = pred_masks[0, organ_idx].cpu().numpy()
            if mask.sum() > 0:
                # Align the mask to the original image coordinates
                if mask.shape != original_img.shape:
                    mask = self._align_mask_to_original(mask, original_img.shape)

                # Create a colored overlay with transparency
                colored_mask = np.zeros((*original_img.shape, 4))
                colored_mask[mask > 0] = (*color[:3], 0.3)
                plt.imshow(
                    colored_mask, extent=[0, original_img.shape[1], original_img.shape[0], 0]
                )

                # Add legend entry for the organ
                organ_name = list(self.organ_map.keys())[
                    list(self.organ_map.values()).index(organ_idx)
                ]
                plt.plot([], [], color=color, label=organ_name, linewidth=3)

        plt.title("Segmentation Overlay")
        plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
        plt.axis("off")

        save_path = self.temp_dir / f"segmentation_{uuid.uuid4().hex[:8]}.png"
        plt.savefig(save_path, bbox_inches="tight", dpi=300)
        plt.close()

        return str(save_path)

    def _run(
        self,
        image_path: str,
        organs: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> Tuple[Dict[str, Any], Dict]:
        """Run segmentation analysis for specified organs."""
        try:
            # Validate and get organ indices
            if organs:
                organs = [o.strip() for o in organs]
                invalid_organs = [o for o in organs if o not in self.organ_map]
                if invalid_organs:
                    raise ValueError(f"Invalid organs specified: {invalid_organs}")
                organ_indices = [self.organ_map[o] for o in organs]
            else:
                organ_indices = list(self.organ_map.values())
                organs = list(self.organ_map.keys())

            # Load and process image
            original_img = skimage.io.imread(image_path)
            if len(original_img.shape) > 2:
                original_img = original_img[:, :, 0]

            img = xrv.datasets.normalize(original_img, 255)
            img = img[None, ...]
            img = self.transform(img)
            img = torch.from_numpy(img)
            img = img.to(self.device)

            # Generate predictions
            with torch.no_grad():
                pred = self.model(img)
            pred_probs = torch.sigmoid(pred)
            pred_masks = (pred_probs > 0.5).float()

            # Save visualization
            viz_path = self._save_visualization(original_img, pred_masks, organ_indices)

            # Compute metrics for selected organs
            results = {}
            for idx, organ_name in zip(organ_indices, organs):
                mask = pred_masks[0, idx].cpu().numpy()
                if mask.sum() > 0:
                    metrics = self._compute_organ_metrics(
                        mask, original_img, float(pred_probs[0, idx].mean().cpu())
                    )
                    if metrics:
                        results[organ_name] = metrics

            output = {
                "segmentation_image_path": viz_path,
                "metrics": {organ: metrics.dict() for organ, metrics in results.items()},
            }

            metadata = {
                "image_path": image_path,
                "segmentation_image_path": viz_path,
                "original_size": original_img.shape,
                "model_size": tuple(img.shape[-2:]),
                "pixel_spacing_mm": self.pixel_spacing_mm,
                "requested_organs": organs,
                "processed_organs": list(results.keys()),
                "analysis_status": "completed",
            }

            return output, metadata

        except Exception as e:
            error_output = {"error": str(e)}
            error_metadata = {
                "image_path": image_path,
                "analysis_status": "failed",
                "error_traceback": traceback.format_exc(),
            }
            return error_output, error_metadata

    async def _arun(
        self,
        image_path: str,
        organs: Optional[List[str]] = None,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> Tuple[Dict[str, Any], Dict]:
        """Async version of _run."""
        return self._run(image_path, organs)


class ChestXRayInput(BaseModel):
    """Input for chest X-ray analysis tools. Only supports JPG or PNG images."""

    image_path: str = Field(
        ..., description="Path to the radiology image file, only supports JPG or PNG images"
    )


class ChestXRayClassifierTool(BaseTool):
    """Tool for automated chest X-ray analysis.

    This tool analyzes chest X-ray images to detect and classify multiple
    potential pathologies. It predicts the likelihood of various conditions
    being present in the image, providing probabilities between 0 and 1.

    The supported pathologies include:
    Atelectasis, Cardiomegaly, Consolidation, Edema, Effusion, Emphysema,
    Enlarged Cardiomediastinum, Fibrosis, Fracture, Hernia, Infiltration,
    Lung Lesion, Lung Opacity, Mass, Nodule, Pleural Thickening, Pneumonia, Pneumothorax.

    Higher values indicate a greater likelihood of the corresponding condition.
    """

    name: str = "chest_xray_classifier"
    description: str = (
        "A tool that analyzes chest X-ray images and predicts the probability of 18 different pathologies. "
        "Input should be the path to a chest X-ray image file. "
        "Output is a dictionary mapping each pathology to a probability value (0 to 1), "
        "where higher values indicate a higher likelihood of the condition being present. "
        "Supported pathologies: Atelectasis, Cardiomegaly, Consolidation, Edema, Effusion, Emphysema, "
        "Enlarged Cardiomediastinum, Fibrosis, Fracture, Hernia, Infiltration, Lung Lesion, "
        "Lung Opacity, Mass, Nodule, Pleural Thickening, Pneumonia, and Pneumothorax."
    )
    args_schema: Type[BaseModel] = ChestXRayInput
    model: Optional[object] = None
    device: Optional[str] = "cuda"
    transform: Optional[torchvision.transforms.Compose] = None

    def __init__(self, model_name: str = "densenet121-res224-all", device: Optional[str] = "cuda"):
        super().__init__()
        self.model = xrv.models.DenseNet(weights=model_name)
        self.model.eval()
        self.device = torch.device(device) if device else "cuda"
        self.model = self.model.to(self.device)
        self.transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop()])

    def _process_image(self, image_path: str) -> torch.Tensor:
        """
        Process the input chest X-ray image for model inference.

        This method loads the image, normalizes it, applies necessary transformations,
        and prepares it as a torch.Tensor for model input.

        Args:
            image_path (str): The file path to the chest X-ray image.

        Returns:
            torch.Tensor: A processed image tensor ready for model inference.

        Raises:
            FileNotFoundError: If the specified image file does not exist.
            ValueError: If the image cannot be properly loaded or processed.
        """
        img = skimage.io.imread(image_path)
        img = xrv.datasets.normalize(img, 255)

        if len(img.shape) > 2:
            img = img[:, :, 0]

        img = img[None, :, :]
        img = self.transform(img)
        img = torch.from_numpy(img).unsqueeze(0)

        img = img.to(self.device)

        return img

    def _run(
        self,
        image_path: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> Tuple[Dict[str, float], Dict]:
        """Classify the chest X-ray image for multiple pathologies.

        Args:
            image_path (str): The path to the chest X-ray image file.
            run_manager (Optional[CallbackManagerForToolRun]): The callback manager for the tool run.

        Returns:
            Tuple[Dict[str, float], Dict]: A tuple containing the classification results
                                           (pathologies and their probabilities from 0 to 1)
                                           and any additional metadata.

        Raises:
            Exception: If there's an error processing the image or during classification.
        """
        try:
            img = self._process_image(image_path)
            with torch.inference_mode():
                preds = self.model(img).cpu()[0]

            output = dict(zip(xrv.datasets.default_pathologies, preds.numpy()))
            metadata = {
                "image_path": image_path,
                "analysis_status": "completed",
                "note": "Probabilities range from 0 to 1, with higher values indicating higher likelihood of the condition.",
            }
            return output, metadata
        except Exception as e:
            return {"error": str(e)}, {
                "image_path": image_path,
                "analysis_status": "failed",
            }

    async def _arun(
        self,
        image_path: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> Tuple[Dict[str, float], Dict]:
        """Asynchronously classify the chest X-ray image for multiple pathologies.

        This method currently calls the synchronous version, as the model inference
        is not inherently asynchronous. For true asynchronous behavior, consider
        using a separate thread or process.

        Args:
            image_path (str): The path to the chest X-ray image file.
            run_manager (Optional[AsyncCallbackManagerForToolRun]): The async callback manager for the tool run.

        Returns:
            Tuple[Dict[str, float], Dict]: A tuple containing the classification results
                                           (pathologies and their probabilities from 0 to 1)
                                           and any additional metadata.

        Raises:
            Exception: If there's an error processing the image or during classification.
        """
        return self._run(image_path)


class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

gemini_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

tavily_tool = TavilySearch(
    max_results=5,
    topic="general"
)

class GemmaMedInput(BaseModel):
    """Input schema for GemmaMedTool."""
    question: str = Field(description="The medical question to answer")
    image_path: Optional[str] = Field(default=None, description="Path to the medical image file (optional)")


class GemmaMedTool(BaseTool):
    """Tool for medical visual question answering and report generation.

    This tool can answer questions related to medical images as well as
    general medical queries without images. It also supports generating
    structured medical reports from the provided information.
    """

    name: str = "medical_qa_tool"
    description: str = (
        "A tool that answers questions related to medical images (e.g., chest X-rays, CT scans, MRIs) "
        "and general medical queries without images. "
        "Additionally, it can generate structured medical reports based on the provided input."
    )
    args_schema: Type[BaseModel] = GemmaMedInput

    # Declare the fields as class attributes
    client: Any = None
    model_name: str = "amsaravi/medgemma-4b-it:q8"
    image_resize: Tuple[int, int] = (224, 224)
    system_prompt: str = "You are a helpful medical assistant specialized in analyzing medical images and answering medical questions."

    def __init__(
        self,
        base_url: str = "http://localhost:11434/v1",
        api_key: str = "ollama",
        model_name: str = "amsaravi/medgemma-4b-it:q8",
        image_resize: Tuple[int, int] = (224, 224),
        system_prompt: str = "You are a helpful medical assistant specialized in analyzing medical images and answering medical questions.",
        **kwargs,
    ):
        """Initialize the GemmaMedTool.

        Args:
            base_url: The base URL for the Ollama API
            api_key: API key for compatibility (usually "ollama" for local Ollama)
            model_name: Name of the MedGemma model to use
            image_resize: Tuple of (width, height) to resize images
            system_prompt: System prompt for the model
        """
        super().__init__(**kwargs)
        self.client = OpenAI(base_url=base_url, api_key=api_key)
        self.model_name = model_name
        self.image_resize = image_resize
        self.system_prompt = system_prompt

    def _encode_image(self, image_path: str) -> str:
        """Encode image to base64 string.

        Args:
            image_path: Path to the image file

        Returns:
            Base64 encoded image string
        """
        try:
            # Open and resize image
            image = Image.open(image_path)
            if self.image_resize:
                image = image.resize(self.image_resize)

            # Save to temporary path if resized
            if self.image_resize:
                temp_path = image_path.replace('.jpg', '_resized.jpg').replace('.png', '_resized.png')
                image.save(temp_path)
                image_path = temp_path

            # Encode to base64
            with open(image_path, "rb") as f:
                img_b64 = base64.b64encode(f.read()).decode("utf-8")

            return img_b64
        except Exception as e:
            raise Exception(f"Error encoding image: {str(e)}")

    def _create_messages(self, question: str, image_path: Optional[str] = None) -> list:
        """Create message list for the API call.

        Args:
            question: The medical question
            image_path: Optional path to medical image

        Returns:
            List of messages for the API
        """
        messages = [
            {"role": "system", "content": self.system_prompt}
        ]

        if image_path:
            img_b64 = self._encode_image(image_path)
            user_message = {
                "role": "user",
                "content": [
                    {"type": "text", "text": question},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{img_b64}"
                        }
                    }
                ]
            }
        else:
            user_message = {
                "role": "user",
                "content": question
            }

        messages.append(user_message)
        return messages

    def _run(
        self,
        question: str,
        image_path: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> Tuple[str, Dict]:
        """Answer a medical question, optionally based on an input image.

        Args:
            question (str): The medical question to answer.
            image_path (Optional[str]): The path to the medical image file (if applicable).
            run_manager (Optional[CallbackManagerForToolRun]): The callback manager for the tool run.

        Returns:
            Tuple[str, Dict]: A tuple containing the model's answer and any additional metadata.

        Raises:
            Exception: If there's an error processing the input or generating the answer.
        """
        try:
            messages = self._create_messages(question, image_path)

            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                temperature=0.2,
                max_tokens=500,
            )

            answer = response.choices[0].message.content

            metadata = {
                "question": question,
                "image_path": image_path,
                "model_name": self.model_name,
                "analysis_status": "completed",
                "has_image": image_path is not None
            }

            return answer, metadata

        except Exception as e:
            error_msg = f"Error generating answer: {str(e)}"
            metadata = {
                "question": question,
                "image_path": image_path,
                "model_name": self.model_name,
                "analysis_status": "failed",
                "has_image": image_path is not None,
                "error": str(e)
            }
            return error_msg, metadata

    async def _arun(
        self,
        question: str,
        image_path: Optional[str] = None,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> Tuple[str, Dict]:
        """Asynchronously answer a medical question, optionally based on an input image.

        This method currently calls the synchronous version. For true async behavior,
        you could use asyncio.to_thread() or implement async OpenAI client calls.

        Args:
            question (str): The medical question to answer.
            image_path (Optional[str]): The path to the medical image file (if applicable).
            run_manager (Optional[AsyncCallbackManagerForToolRun]): The async callback manager for the tool run.

        Returns:
            Tuple[str, Dict]: A tuple containing the model's answer and any additional metadata.

        Raises:
            Exception: If there's an error processing the input or generating the answer.
        """
        return self._run(question, image_path)


# Initialize the tool
gemma_med_tool = GemmaMedTool()
chest_xray_segmentation_tool = ChestXRaySegmentationTool()
chest_xray_classifier = ChestXRayClassifierTool()
# Tool creation
tools = [tavily_tool, gemma_med_tool, chest_xray_classifier]
# Tool binding
llm_with_tools = gemini_llm.bind_tools(tools)

# graph nodes
def chat_node(state: ChatState):
    """LLM node that may answer or request a tool call."""
    messages = state['messages']
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

tool_node = ToolNode(tools)

# graph structure
graph = StateGraph(ChatState)
graph.add_node("chat_node", chat_node)
graph.add_node("tools", tool_node)
graph.add_edge(START, "chat_node")

# If the LLM asked for a tool, go to ToolNode; else finish
graph.add_conditional_edges("chat_node", tools_condition)

graph.add_edge("tools", "chat_node")

checkpointer = InMemorySaver()
chatbot = graph.compile(checkpointer=checkpointer)

Overwriting helper.py


In [9]:
%%writefile app.py
import streamlit as st
import hashlib
import json
import os
import tempfile
from pathlib import Path
from langchain_core.messages import HumanMessage
from PIL import Image
import base64

from helper import chatbot, chest_xray_segmentation_tool

# Authentication functions
def hash_password(password):
    """Hash password using SHA256"""
    return hashlib.sha256(password.encode()).hexdigest()

def load_users():
    """Load users from JSON file"""
    users_file = "users.json"
    if os.path.exists(users_file):
        with open(users_file, 'r') as f:
            return json.load(f)
    return {}

def save_users(users):
    """Save users to JSON file"""
    users_file = "users.json"
    with open(users_file, 'w') as f:
        json.dump(users, f)

def signup(username, password):
    """Register a new user"""
    users = load_users()
    if username in users:
        return False, "Username already exists"

    users[username] = {
        "password": hash_password(password),
        "message_history": []
    }
    save_users(users)
    return True, "User registered successfully"

def login(username, password):
    """Authenticate user"""
    users = load_users()
    if username not in users:
        return False, "Username not found"

    if users[username]["password"] == hash_password(password):
        return True, "Login successful"
    return False, "Invalid password"

def save_message_history(username, messages):
    """Save user's message history"""
    users = load_users()
    if username in users:
        users[username]["message_history"] = messages
        save_users(users)

def load_message_history(username):
    """Load user's message history"""
    users = load_users()
    if username in users:
        return users[username]["message_history"]
    return []

def save_uploaded_image(uploaded_file):
    """Save uploaded image to temporary file and return path"""
    if uploaded_file is not None:
        # Create a temporary file
        temp_dir = tempfile.mkdtemp()
        temp_path = os.path.join(temp_dir, uploaded_file.name)

        # Save the uploaded file
        with open(temp_path, "wb") as f:
            f.write(uploaded_file.getbuffer())

        return temp_path
    return None

def perform_chest_xray_segmentation(image_path):
    """Perform chest X-ray segmentation and return segmented image path"""
    try:
        result, metadata = chest_xray_segmentation_tool._run(image_path=image_path)
        return metadata.get("segmentation_image_path"), result
    except Exception as e:
        st.error(f"Error during segmentation: {str(e)}")
        return None, None

def display_image_with_question(image_path, question, segmentation_path=None):
    """Display image alongside the question, with optional segmentation"""
    if segmentation_path:
        # Show original and segmented images side by side
        st.subheader("Original and Segmented Images")
        col1, col2 = st.columns(2)

        with col1:
            if image_path and os.path.exists(image_path):
                image = Image.open(image_path)
                st.image(image, caption="Original Image", use_container_width=True)

        with col2:
            if os.path.exists(segmentation_path):
                seg_image = Image.open(segmentation_path)
                st.image(seg_image, caption="Segmented Image", use_container_width=True)

        st.markdown(f"**Question:** {question}")
    else:
        # Original layout for non-segmented images
        col1, col2 = st.columns([1, 2])

        with col1:
            if image_path and os.path.exists(image_path):
                image = Image.open(image_path)
                st.image(image, caption="Uploaded Image", use_container_width=True)

        with col2:
            st.text(question)

def is_chest_xray_image(image_path):
    """Simple heuristic to determine if image might be a chest X-ray"""
    # You can enhance this logic based on your needs
    # For now, we'll assume any uploaded medical image could be a chest X-ray
    return True

# Main App
def main():
    st.set_page_config(page_title="Medical Visual Chatbot", page_icon="🏥", layout="wide")

    # Initialize session state
    if 'authenticated' not in st.session_state:
        st.session_state['authenticated'] = False
    if 'username' not in st.session_state:
        st.session_state['username'] = None
    if 'message_history' not in st.session_state:
        st.session_state['message_history'] = []

    # Header
    st.title("🏥 Medical Visual Chatbot with X-ray Segmentation")
    st.markdown("---")

    # Authentication Section
    if not st.session_state['authenticated']:
        st.subheader("Please Login or Sign Up")

        tab1, tab2 = st.tabs(["Login", "Sign Up"])

        with tab1:
            with st.form("login_form"):
                username = st.text_input("Username")
                password = st.text_input("Password", type="password")
                login_button = st.form_submit_button("Login")

                if login_button:
                    if username and password:
                        success, message = login(username, password)
                        if success:
                            st.session_state['authenticated'] = True
                            st.session_state['username'] = username
                            st.session_state['message_history'] = load_message_history(username)
                            st.success(message)
                            st.rerun()
                        else:
                            st.error(message)
                    else:
                        st.error("Please enter both username and password")

        with tab2:
            with st.form("signup_form"):
                new_username = st.text_input("Choose Username")
                new_password = st.text_input("Choose Password", type="password")
                confirm_password = st.text_input("Confirm Password", type="password")
                signup_button = st.form_submit_button("Sign Up")

                if signup_button:
                    if new_username and new_password and confirm_password:
                        if new_password == confirm_password:
                            success, message = signup(new_username, new_password)
                            if success:
                                st.success(message)
                            else:
                                st.error(message)
                        else:
                            st.error("Passwords do not match")
                    else:
                        st.error("Please fill in all fields")

    # Main Chat Interface
    else:
        # Sidebar with user info and logout
        with st.sidebar:
            st.write(f"**Logged in as:** {st.session_state['username']}")
            if st.button("Logout"):
                # Save message history before logout
                save_message_history(st.session_state['username'], st.session_state['message_history'])
                st.session_state['authenticated'] = False
                st.session_state['username'] = None
                st.session_state['message_history'] = []
                st.rerun()

            st.markdown("---")
            st.markdown("### About")
            st.markdown("This chatbot can answer medical questions and analyze medical images using MedGemma AI. It also provides automatic chest X-ray segmentation.")

            # Clear chat button
            if st.button("Clear Chat History"):
                st.session_state['message_history'] = []
                save_message_history(st.session_state['username'], [])
                st.rerun()

        # Main chat area
        st.subheader("Chat with Medical AI")

        # Display chat history
        chat_container = st.container()
        with chat_container:
            for message in st.session_state['message_history']:
                with st.chat_message(message['role']):
                    if message['role'] == 'user' and 'image_path' in message:
                        # Display image with question, including segmentation if available
                        segmentation_path = message.get('segmentation_path')
                        display_image_with_question(
                            message.get('image_path'),
                            message['content'],
                            segmentation_path
                        )
                    else:
                        st.markdown(message['content'])

        # Input section
        st.markdown("---")
        col1, col2 = st.columns([3, 1])

        with col1:
            user_input = st.text_input("Ask a medical question:", key="user_question")

        with col2:
            uploaded_file = st.file_uploader(
                "Upload medical image (optional)",
                type=['png', 'jpg', 'jpeg', 'dicom'],
                key="image_upload"
            )

        # Submit button
        if st.button("Send", type="primary"):
            if user_input.strip():
                # Save uploaded image if provided
                image_path = None
                segmentation_path = None
                segmentation_result = None

                if uploaded_file is not None:
                    image_path = save_uploaded_image(uploaded_file)

                    # Perform segmentation if it's a chest X-ray
                    if image_path and is_chest_xray_image(image_path):
                        with st.spinner("Processing chest X-ray segmentation..."):
                            segmentation_path, segmentation_result = perform_chest_xray_segmentation(image_path)

                # Add user message to history
                user_message = {
                    'role': 'user',
                    'content': user_input,
                }
                if image_path:
                    user_message['image_path'] = image_path
                if segmentation_path:
                    user_message['segmentation_path'] = segmentation_path

                st.session_state['message_history'].append(user_message)

                # Display user message
                with st.chat_message('user'):
                    if image_path:
                        display_image_with_question(image_path, user_input, segmentation_path)
                    else:
                        st.markdown(user_input)

                # Generate response using your chatbot
                try:
                    with st.spinner("Thinking..."):
                        # Configuration for your chatbot
                        CONFIG = {'configurable': {'thread_id': st.session_state['username']}}

                        # Create the message for the chatbot
                        if image_path:
                            # Include image path and segmentation info in the prompt
                            prompt_parts = [f"Please analyze the medical image at path: {image_path}"]

                            # if segmentation_path and segmentation_result:
                            #     prompt_parts.append(f"Segmentation analysis: {segmentation_result}")
                            #     prompt_parts.append("A segmented version of the image has been generated for better analysis.")

                            prompt_parts.append(f"Question: {user_input}")
                            prompt_with_image = "\n\n".join(prompt_parts)

                            human_message = HumanMessage(content=prompt_with_image)
                        else:
                            human_message = HumanMessage(content=user_input)

                        # Get response from chatbot
                        response = chatbot.invoke({'messages': [human_message]}, config=CONFIG)
                        ai_message = response['messages'][-1].content

                        # Add segmentation info to AI response if available
                        # if segmentation_result:
                        #     ai_message = f"**Segmentation Analysis:**\n{segmentation_result}\n\n**Detailed Analysis:**\n{ai_message}"

                        # Add AI response to history
                        st.session_state['message_history'].append({
                            'role': 'assistant',
                            'content': ai_message
                        })

                        # Display AI response
                        with st.chat_message('assistant'):
                            st.markdown(ai_message)

                        # Save message history
                        save_message_history(st.session_state['username'], st.session_state['message_history'])

                except Exception as e:
                    st.error(f"Error generating response: {str(e)}")
                    # You might want to log this error
                    print(f"Chatbot error: {str(e)}")

                # Clear the input
                st.rerun()
            else:
                st.warning("Please enter a question.")

        # # Instructions
        # with st.expander("ℹ️ How to use"):
        #     st.markdown("""
        #     1. **Text Questions**: Type any medical question in the text input
        #     2. **Image Analysis**: Upload a medical image (X-ray, CT scan, etc.) and ask questions about it
        #     3. **Automatic Segmentation**: Chest X-rays will be automatically segmented for better analysis
        #     4. **Examples**:
        #        - "What does this X-ray show?"
        #        - "Explain the symptoms of diabetes"
        #        - "What abnormalities can you see in this CT scan?"
        #        - "Analyze the lung regions in this chest X-ray"

        #     **Note**: This tool is for educational purposes only and should not replace professional medical advice.

        #     **New Feature**: When you upload a chest X-ray, the system will automatically generate a segmented version
        #     highlighting different anatomical regions for better analysis.
        #     """)

if __name__ == "__main__":
    main()

Overwriting app.py


In [10]:
from pyngrok import ngrok

ngrok_key = ngrok_key
port = 8501

ngrok.set_auth_token(ngrok_key)
ngrok.connect(port).public_url

'https://cbab8cb4cdd2.ngrok-free.app'

In [11]:
!rm -rf logs.txt && streamlit run app.py &>/content/logs.txt